## Reinforcement Learning Value iteration to find the optimal policy

In [1]:
import numpy as np
import math
from tqdm import tqdm
import random

In [2]:
S = [[0,1,2,3], [4,5,6,7], [8,9,10,11], [12,13,14,0]]   #0=Terminals
A = [0, 1, 2, 3]      #0='up', 1='down', 2='right', 3='left'
num_states = 14+2     #including terminals
actions = []

for i in range(0, num_states):
    actions.append([0,1,2,3])

actions     #possible moves/actions

[[0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3]]

In [38]:
#Considering probability 1 for a transition
#For example, if action is DOWN and state is 1, there are probabilities of going DOWN or LEFT or RIGHT
#So probability of going DOWN could be 0.8, LEFT could be 0.1 and RIGHT could be 0.1
#But here we're considering that if action is DOWN, probability of the transition or going DOWN is 1 
#because the model always picks up the highest probability

flat_list = [item for sublist in actions for item in sublist] #flatten the list of lists
num_actions = len(set(flat_list))
trans_prob = [[[0 for i in range(num_states)] for j in range(num_states)] for k in range(num_actions)]

#0=up
trans_prob[0][0][0] = 1   #all actions in this terminal state transition to itself with probability 1
trans_prob[0][1][1] = 1
trans_prob[0][2][2] = 1
trans_prob[0][3][3] = 1
trans_prob[0][4][0] = 1   #0,4,0
trans_prob[0][5][1] = 1
trans_prob[0][6][2] = 1
trans_prob[0][7][3] = 1
trans_prob[0][8][4] = 1
trans_prob[0][9][5] = 1
trans_prob[0][10][6] = 1
trans_prob[0][11][7] = 1
trans_prob[0][12][8] = 1
trans_prob[0][13][9] = 1
trans_prob[0][14][10] = 1
trans_prob[0][15][15] = 1 #all actions in this terminal state transition to itself with probability 1

#1=down
trans_prob[1][0][0] = 1   #all actions in this terminal state transition to itself with probability 1
trans_prob[1][1][5] = 1
trans_prob[1][2][6] = 1
trans_prob[1][3][7] = 1
trans_prob[1][4][8] = 1
trans_prob[1][5][9] = 1
trans_prob[1][6][10] = 1
trans_prob[1][7][11] = 1
trans_prob[1][8][12] = 1
trans_prob[1][9][13] = 1
trans_prob[1][10][14] = 1
trans_prob[1][11][15] = 1  #1,11,0
trans_prob[1][12][12] = 1
trans_prob[1][13][13] = 1
trans_prob[1][14][14] = 1
trans_prob[1][15][15] = 1  #all actions in this terminal state transition to itself with probability 1

#2=right
trans_prob[2][0][0] = 1    #all actions in this terminal state transition to itself with probability 1
trans_prob[2][1][2] = 1
trans_prob[2][2][3] = 1
trans_prob[2][3][3] = 1
trans_prob[2][4][5] = 1
trans_prob[2][5][6] = 1
trans_prob[2][6][7] = 1
trans_prob[2][7][7] = 1
trans_prob[2][8][9] = 1
trans_prob[2][9][10] = 1
trans_prob[2][10][11] = 1
trans_prob[2][11][11] = 1
trans_prob[2][12][13] = 1
trans_prob[2][13][14] = 1
trans_prob[2][14][15] = 1  #2,14,0
trans_prob[2][15][15] = 1  #all actions in this terminal state transition to itself with probability 1

#3=left
trans_prob[3][0][0] = 1    #all actions in this terminal state transition to itself with probability 1
trans_prob[3][1][0] = 1    #3,1,0
trans_prob[3][2][1] = 1
trans_prob[3][3][2] = 1
trans_prob[3][4][4] = 1
trans_prob[3][5][4] = 1
trans_prob[3][6][5] = 1
trans_prob[3][7][6] = 1
trans_prob[3][8][8] = 1
trans_prob[3][9][8] = 1
trans_prob[3][10][9] = 1
trans_prob[3][11][10] = 1
trans_prob[3][12][12] = 1
trans_prob[3][13][12] = 1
trans_prob[3][14][13] = 1
trans_prob[3][15][15] = 1  #all actions in this terminal state transition to itself with probability 1

trans_prob

[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],
 [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0,

In [4]:
#initializing with lower reward value
rewrds = [[-2 for i in range(num_actions)] for j in range(num_states)] 

#given that all valid transitions have -1 as reward
for i in range(0, num_states):
    for j in range(0, len(actions[i])):
        rewrds[i][actions[i][j]] = -1

##all actions in the terminal state transition to itself without any reward or lowest reward
#set lower reward for terminals
rewrds[0] = [-2,-2,-2,-2]    
rewrds[15] = [-2,-2,-2,-2]

#set lower reward for out of boundary actions
#0=up
rewrds[1][0] = -2
rewrds[2][0] = -2
rewrds[3][0] = -2
rewrds[4][3] = -2

#1=down
rewrds[12][1] = -2
rewrds[13][1] = -2
rewrds[14][1] = -2

#2=right
rewrds[3][2] = -2
rewrds[7][2] = -2
rewrds[11][2] = -2


#3=left
rewrds[4][3] = -2
rewrds[8][3] = -2
rewrds[12][3] = -2

rewrds

[[-2, -2, -2, -2],
 [-2, -1, -1, -1],
 [-2, -1, -1, -1],
 [-2, -1, -2, -1],
 [-1, -1, -1, -2],
 [-1, -1, -1, -1],
 [-1, -1, -1, -1],
 [-1, -1, -2, -1],
 [-1, -1, -1, -2],
 [-1, -1, -1, -1],
 [-1, -1, -1, -1],
 [-1, -1, -2, -1],
 [-1, -2, -1, -2],
 [-1, -2, -1, -1],
 [-1, -2, -1, -1],
 [-2, -2, -2, -2]]

In [32]:
def VI(giNumStates, gvActions, gvTrp, gvRews, gdDiscountFactor):
    
    t_dEpsilon = 0.000001
    tmpv_vals = [0]*giNumStates
    flat_list = [item for sublist in gvActions for item in sublist]
    t_iNumActions = len(set(flat_list))
    t_iCtr = 0
    
    while True:
        currentv_vals = [0]*giNumStates
        t_qValues = [[float("-inf") for a in range(t_iNumActions)] for s in range(giNumStates)]
        t_qValues[0] = [0, 0, 0, 0]
        t_qValues[giNumStates - 1] = [0, 0, 0, 0]
        
        t_vPol = [float("-inf") for s in range(giNumStates)]
        for s in range(giNumStates):
            if(s > 0 and s < giNumStates - 1):
                for a in gvActions[s]:
                    t_qValues[s][a] = gvRews[s][a]
                    for ss in range(giNumStates):
                        if(ss > 0 and ss < giNumStates - 1):
                            t_qValues[s][a] += gdDiscountFactor*gvTrp[a][s][ss]*tmpv_vals[ss]
        
        currentv_vals = np.amax(np.around(t_qValues, 3), axis = 1)
        t_vPol = np.argmax(np.around(t_qValues, 3), axis = 1)        
        
        
        if np.linalg.norm(np.array(tmpv_vals) - np.array(currentv_vals)) < t_dEpsilon:
            break
                
        tmpv_vals = currentv_vals  
        t_iCtr = t_iCtr + 1
        if(t_iCtr == 2): prev_pol = t_vPol
                            
    t_vPol[0] = -1
    t_vPol[giNumStates - 1] = -1
    prev_pol[0] = -1
    prev_pol[giNumStates - 1] = -1
    
    return tmpv_vals, t_vPol, t_iCtr, prev_pol

In [33]:
import time

discount_factor = 1 #undiscounted

start = time.time()
state_values, opt_policy, iter_count, prev_pol = VI(num_states, actions, trans_prob, rewrds, discount_factor)
end = time.time()

In [34]:
print("VALUE ITERATION")
print()
print("Number of iterations required to achieve optimal policy: ", iter_count)
print()
print("Time taken: ", round(end - start, 4))
print()

print("State Values:")
print("", state_values[0], " | ", state_values[1], " | ", state_values[2], " | ", state_values[3])
print("-----------------------")
print(state_values[4], " | ", state_values[5], " | ", state_values[6], " | ", state_values[7])
print("-----------------------")
print(state_values[8], " | ", state_values[9], " | ", state_values[10], " | ", state_values[11])
print("-----------------------")
print(state_values[12], " | ", state_values[13], " | ", state_values[14], " | ", state_values[15])
print("-----------------------")
print()

print("Optimal Policy: (-1 represents No Policy for Terminal States)")
print("Policy-0: UP, 1:DOWN, 2:RIGHT, 3:LEFT")
print()
for i in range(0, len(opt_policy)):
    print("State: ", i, "     Policy: ", opt_policy[i])

print()

actions_taken = ["" for x in range(len(opt_policy))]

for i in range(0, len(opt_policy)):
    if(opt_policy[i] == -1): actions_taken[i] = "END"
    if(opt_policy[i] == 0): actions_taken[i] = "^"
    if(opt_policy[i] == 1): actions_taken[i] = "v"
    if(opt_policy[i] == 2): actions_taken[i] = ">"
    if(opt_policy[i] == 3): actions_taken[i] = "<"
        
print("Actions")
print(actions_taken[0], " | ", actions_taken[1], " | ", actions_taken[2], " | ", actions_taken[3])
print("-----------------------")
print(" ", actions_taken[4], " | ", actions_taken[5], " | ", actions_taken[6], " | ", actions_taken[7])
print("-----------------------")
print(" ", actions_taken[8], " | ", actions_taken[9], " | ", actions_taken[10], " | ", actions_taken[11])
print("-----------------------")
print(" ", actions_taken[12], " | ", actions_taken[13], " | ", actions_taken[14], " | ", actions_taken[15])
print("-----------------------")
print()

print("Policy from previous iteration (before last iteration):")
print()
for i in range(0, len(prev_pol)):
    print("State: ", i, "     Policy: ", prev_pol[i])

VALUE ITERATION

Number of iterations required to achieve optimal policy:  3

Time taken:  0.002

State Values:
 0  |  -1  |  -2  |  -3
-----------------------
-1  |  -2  |  -3  |  -2
-----------------------
-2  |  -3  |  -2  |  -1
-----------------------
-3  |  -2  |  -1  |  0
-----------------------

Optimal Policy: (-1 represents No Policy for Terminal States)
Policy-0: UP, 1:DOWN, 2:RIGHT, 3:LEFT

State:  0      Policy:  -1
State:  1      Policy:  3
State:  2      Policy:  3
State:  3      Policy:  1
State:  4      Policy:  0
State:  5      Policy:  0
State:  6      Policy:  0
State:  7      Policy:  1
State:  8      Policy:  0
State:  9      Policy:  0
State:  10      Policy:  1
State:  11      Policy:  1
State:  12      Policy:  0
State:  13      Policy:  2
State:  14      Policy:  2
State:  15      Policy:  -1

Actions
END  |  <  |  <  |  v
-----------------------
  ^  |  ^  |  ^  |  v
-----------------------
  ^  |  ^  |  v  |  v
-----------------------
  ^  |  >  |  >  |  END


## Q-learning algorithm

In [39]:
def QLearning(giNumStates, giNumActions, gvActions, gvTrp, gvRews, gdDiscountFactor, step_size=0.5):
        
    d_epsilon = 0.9 # randomly select an action with this prob
    episodes = 500 # episodes for each run    
    runs = 10 # perform 10 independent runs
    print("Number of Iterations: ", runs)
    print("Number of Episodes per Iteration: ", episodes)
    print()

    rewards_q_learning = np.zeros(episodes)
    for r in tqdm(range(runs)):
        print("begin-run", r)
        
        q_value = np.zeros((giNumStates,giNumActions))
        
        for i in range(0, episodes):
            state = 1 
            rewards = 0
            
            while state > 0 and state < giNumStates - 1: #runs until terminal states have reached
                if (np.random.random() < d_epsilon):
                    action = np.random.choice(gvActions[state])
                else:
                    values_ = q_value[state, :]
                    action = np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

                next_state = np.random.choice(np.arange(len(gvTrp[action][state])), p = gvTrp[action][state])
                reward = gvRews[state][action]
                
                rewards += reward

                #Q-Learning update
                q_value[state, action] += step_size * (reward + gdDiscountFactor * np.max(q_value[next_state, :])-q_value[state, action])

                state = next_state
            
            rewards_q_learning[i] += rewards
    
        if(r == runs-2): q_value_prev = q_value;
        print("end-run", r)
        
    rewards_q_learning /= runs
    print("Rewards: ")
    print(rewards_q_learning)
    
    optimal_policy = {}
    prev_opt_pol = {}
    for s in range(giNumStates):
        optimal_policy[s] = np.argmax(q_value[s, :])
        prev_opt_pol[s] = np.argmax(q_value_prev[s, :])
                    
    optimal_policy[0] = -1
    optimal_policy[giNumStates - 1] = -1
    prev_opt_pol[0] = -1
    prev_opt_pol[giNumStates - 1] = -1
    
    print("Q-Values:")
    print(q_value)
    
    return q_value, optimal_policy, prev_opt_pol
    

In [40]:
print("Q-LEARNING")
print()

discount_factor = 1 #undiscounted

start = time.time()
q_values, opt_pol, prev_opt_pol = QLearning(num_states, num_actions, actions, trans_prob, rewrds, discount_factor)
end = time.time()

Q-LEARNING

Number of Iterations:  10
Number of Episodes per Iteration:  500



  0%|          | 0/10 [00:00<?, ?it/s]

begin-run 0
end-run 0


 10%|█         | 1/10 [00:00<00:01,  7.21it/s]

begin-run 1
end-run 1


...

begin-run 8
end-run 8


 90%|█████████ | 9/10 [00:01<00:00,  7.63it/s]

begin-run 9
end-run 9


100%|██████████| 10/10 [00:01<00:00,  7.30it/s]


Rewards: 
[ -9.8 -25.6 -21.  -20.6  -9.6 -17.  -10.8 -11.  -11.  -13.8  -5.6 -10.
 -17.8 -12.6 -13.6 -11.6 -11.4 -11.2  -7.  -12.2 -13.2  -7.4  -7.6 -17.4
 -11.   -7.6  -4.4 -22.2  -6.2  -9.4 -17.6 -11.6 -19.8  -4.6  -9.4 -17.8
 -12.6 -14.6  -8.  -11.   -9.   -3.4 -11.8  -6.6  -6.6 -13.2 -17.8 -10.8
 -16.4  -5.6 -16.   -4.2 -13.2  -5.2  -9.   -9.   -9.2 -18.2 -22.6  -7.8
  -7.2 -12.6  -9.4  -6.2  -9.4 -25.8 -17.6 -15.4  -8.6  -4.6  -7.8  -2.2
  -8.6 -22.8  -5.2  -9.8  -6.8 -15.  -10.  -13.6  -9.8  -5.  -12.2 -11.4
 -11.   -5.6 -16.4  -8.8 -14.6  -4.6 -12.6 -14.  -10.6 -14.8  -5.8 -10.8
 -10.   -9.2  -7.8 -16.2  -8.8 -10.8  -6.2  -8.2  -8.8 -10.   -7.2 -12.4
 -10.6 -10.4  -9.6  -4.   -7.  -16.  -13.   -6.2 -10.4 -14.2  -7.4 -13.4
  -3.  -15.6 -17.6  -4.4 -10.  -10.2  -7.   -3.4 -15.  -14.4 -11.6  -7.2
  -7.8 -12.2  -7.6  -6.2 -11.6  -7.2  -9.2 -14.   -4.6 -11.8  -8.8 -13.8
  -2.8 -24.6 -14.8  -9.6  -9.8  -4.6  -8.6  -6.2 -12.6  -6.2 -15.6  -3.6
  -7.6  -8.   -3.6 -14.8  -8.2 -13.8 -11. 

In [41]:
print("Time taken: ", round(end - start, 4))
print()

print("Optimal Policy: (-1 represents No Policy for Terminal States)")
print("Policy-0: UP, 1:DOWN, 2:RIGHT, 3:LEFT")
for key,val in opt_pol.items():
    print("State: ", key, "     Policy: ", val)

print()

actions_taken = ["" for x in range(len(opt_pol))]

for i in range(0, len(opt_pol)):
    if(opt_pol[i] == -1): actions_taken[i] = "END"
    if(opt_pol[i] == 0): actions_taken[i] = "^"
    if(opt_pol[i] == 1): actions_taken[i] = "v"
    if(opt_pol[i] == 2): actions_taken[i] = ">"
    if(opt_pol[i] == 3): actions_taken[i] = "<"
        
print("Actions")
print(actions_taken[0], " | ", actions_taken[1], " | ", actions_taken[2], " | ", actions_taken[3])
print("-----------------------")
print(" ", actions_taken[4], " | ", actions_taken[5], " | ", actions_taken[6], " | ", actions_taken[7])
print("-----------------------")
print(" ", actions_taken[8], " | ", actions_taken[9], " | ", actions_taken[10], " | ", actions_taken[11])
print("-----------------------")
print(" ", actions_taken[12], " | ", actions_taken[13], " | ", actions_taken[14], " | ", actions_taken[15])
print("-----------------------")
print()

print("Policy from previous iteration (before last iteration):")
for key,val in prev_opt_pol.items():
    print("State: ", key, "     Policy: ", val)

Time taken:  1.3374

Optimal Policy: (-1 represents No Policy for Terminal States)
Policy-0: UP, 1:DOWN, 2:RIGHT, 3:LEFT
State:  0      Policy:  -1
State:  1      Policy:  3
State:  2      Policy:  3
State:  3      Policy:  1
State:  4      Policy:  0
State:  5      Policy:  0
State:  6      Policy:  0
State:  7      Policy:  1
State:  8      Policy:  0
State:  9      Policy:  2
State:  10      Policy:  1
State:  11      Policy:  1
State:  12      Policy:  2
State:  13      Policy:  2
State:  14      Policy:  2
State:  15      Policy:  -1

Actions
END  |  <  |  <  |  v
-----------------------
  ^  |  ^  |  ^  |  v
-----------------------
  ^  |  >  |  v  |  v
-----------------------
  >  |  >  |  >  |  END
-----------------------

Policy from previous iteration (before last iteration):
State:  0      Policy:  -1
State:  1      Policy:  3
State:  2      Policy:  3
State:  3      Policy:  1
State:  4      Policy:  0
State:  5      Policy:  0
State:  6      Policy:  1
State:  7      Polic